In [1]:
# # uncomment if needed
# !pip3 install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/3e/62/eb702237833c2f49bb7f3a4e6f49807ec213c2539ff8d9bddb2e13aa3690/xgboost-2.0.2-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.8 MB/s eta 0:00:0000:0100:01


In [2]:
# preprocessing 
import pandas as pd
import os
import csv
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

# xgboost model 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Load data

In [3]:
model_data_df = pickle.load(open('../data/model_data.pkl', 'rb'))
model_data_df

,race,sample_data,label
0,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Metaplastic carcinoma, NOS"
1,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Metaplastic carcinoma, NOS"
2,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
3,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
4,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
...,...,...,...
1495,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1496,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1497,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1498,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"


# Data preprocessing

In [22]:
# One-hot encoding 
encoded_df = model_data_df[["race", "sample_data"]]
encoded_df["label"] = pd.factorize(model_data_df.label)[0]
encoded_df


,race,sample_data,label
0,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",0
1,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",0
2,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
3,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
4,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
...,...,...,...
1495,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
1496,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
1497,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1
1498,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...",1


In [23]:
# Turn into Binary Classification
encoded_df = encoded_df[encoded_df.label.isin([0, 1])]

In [24]:
# Filter Data Frames by Race 
white_df = encoded_df[encoded_df["race"] == "white"]
black_df = encoded_df[encoded_df["race"] == "black or african american"]
asian_df = encoded_df[encoded_df["race"] == "asian"]
prognosis_names = encoded_df.label.unique()

# Train Test Split
def df_split(df): 
    split_index = int(0.7 * len(df)) 
    return df.iloc[:split_index], df.iloc[split_index:]

train_white_df, test_white_df = df_split(white_df) 
train_black_df, test_black_df = df_split(black_df) 
train_asian_df, test_asian_df = df_split(asian_df)

# Concatenate Final Sets 
train_df = train_white_df
# train_df = pd.concat([train_white_df, train_black_df, train_asian_df])

In [25]:
# # Prepare Model Data
# pca = PCA(n_components=25) 
# prognosis_names = encoded_df.label.unique()

# # def get_x_y(df): 
#     # x = np.array([pca.fit_transform(np.transpose(data)) for data in df.sample_data.values]) # shrink to 26 values
#     # y = df.filter(prognosis_names).values
#     # return x, y
# def get_x_y(df): 
#     x_init = np.array([pca.fit_transform(np.transpose(data)) for data in df.sample_data.values]) # shrink to 26 values
#     x = x_init.reshape(x_init.shape[0], -1)
#     y = df.label.values
#     return x, y

# def train_val_split(data):
#     x, y = data
#     return train_test_split(x, y, test_size=0.15, random_state=42)

# # Split 
# x_train, x_val, y_train, y_val = train_val_split(get_x_y(train_df)) # training data

# x_train_black, y_train_black = get_x_y(train_black_df)
# x_train_asian, y_train_asian = get_x_y(train_asian_df)

# x_test_white, y_test_white = get_x_y(test_white_df)
# x_test_black, y_test_black = get_x_y(test_black_df)
# x_test_asian, y_test_asian = get_x_y(test_asian_df)
# Prepare Model Data --> [takes 2 min.]
pca = PCA(n_components=1)

def get_x_y(df): 
    x_init = np.array([pca.fit_transform(np.transpose(data)) for data in df.sample_data.values]) # shrink to 26 values
    x = x_init.reshape(x_init.shape[0], -1)
    y = df.label.values
    return x, y

def train_val_split(data):
    # returns: x_train, x_val, y_train, y_val
    x, y = data
    return train_test_split(x, y, test_size=0.15, random_state=42)


##### Main Code #########
x_test_white, y_test_white = get_x_y(test_white_df)
x_test_black, y_test_black = get_x_y(test_black_df)
x_test_asian, y_test_asian = get_x_y(test_asian_df)
x_train, x_val, y_train, y_val = train_val_split(get_x_y(train_df)) # training data

im here 1 


In [26]:
len(encoded_df.label.unique())

2

In [27]:
# Find number of classes for the multi-class classifier
num_class = len(encoded_df.label.unique())

# Create instance of XGBClassifier
xgb_clf = xgb.XGBClassifier(objective='multi:softprob', num_class=num_class, n_estimators=100, seed=42)

# Reshape y_train and y_val if they are not in the correct format
# The labels need to be in a 1D array for XGBoost
if len(y_train.shape) > 1 and y_train.shape[1] > 1:
    y_train = y_train.argmax(axis=1)
if len(y_val.shape) > 1 and y_val.shape[1] > 1:
    y_val = y_val.argmax(axis=1)


# Fit model and predict outcomes

In [28]:
# Fit model
xgb_clf.fit(x_train, y_train)

# Predict y 
y_pred = xgb_clf.predict(x_val)

In [36]:
y_pred_converted = [np.where(x==1)[0][0] for x in y_pred]

In [ ]:
# Model evaluation

In [40]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred_converted)
print(f"Validation Accuracy: {accuracy}")
# print(classification_report(y_val, y_pred, target_names=prognosis_names[:2]))

Validation Accuracy: 0.958904109589041
